# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load city list from weatherpy challenge
# Print df results
file = pd.read_csv("city_data.csv")
file.dropna()
file.head()

,City ID,Date,Country,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloud Cover
0,0,2020-04-09 13:46:05,US,Hilo,19.73,-155.09,71.60,78,5.06,1
1,1,2020-04-09 13:48:33,CA,Aklavik,68.22,-135.01,-2.20,84,3.36,75
2,2,2020-04-09 13:46:06,ZA,Cape Town,-33.93,18.42,82.40,14,11.41,100
3,3,2020-04-09 13:48:33,CV,São Filipe,14.90,-24.50,80.08,53,14.81,0
4,4,2020-04-09 13:48:33,LY,Benghazi,32.12,20.07,61.05,53,13.38,0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Set Gmaps with the G-key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' from DF into lists
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [4]:
# Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heat layer over map 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Set up what my ideal weather would be
nice_temp=file.loc[(file['Max Temp']>=60)&(file['Max Temp']<=85)]
nice_wind=nice_temp.loc[(nice_temp['Wind Speed']<=5)]
nice_weather=nice_wind.loc[(nice_wind['Cloud Cover']==0)]
nice_weather.head()

,City ID,Date,Country,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloud Cover
67,134,2020-04-09 13:48:39,CN,Jingdezhen,29.29,117.21,63.05,61,3.22,0
229,296,2020-04-09 13:49:43,CN,Chaozhou,23.67,116.64,66.20,63,4.47,0
264,331,2020-04-09 13:49:46,AU,Broken Hill,-31.95,141.43,60.80,59,3.36,0
298,365,2020-04-09 13:49:50,AR,Concepción del Uruguay,-32.48,-58.24,64.00,64,1.99,0
335,402,2020-04-09 13:49:53,UA,Poroshkovo,48.67,22.75,73.40,17,2.24,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create hotel file
hotel_df = nice_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# parameters
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}


In [7]:
# Use the stored locations from city_data to identify hotels
for index, row in hotel_df.iterrows():
    
    # latitudes and longitudes from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAACtR-EM4lWEr_Pphe8bAgBiu-PZKiixzyQV0dZBLFeTQaKzEVtjDjSRJ8zKpIEpzgG9IrDmuz65T47WmuO6uXQEZBcJLP9j5BzKwblXS9OsO0rRnSM70DpTZ4GCBSXJrPXrrE3pj_f2TJO1adVL7GI5w6zV8d-WoWXgxYDhPFyACdFya79OeB9TFoe51MpqysOLFWgvrPQh7H73tgx9C1jhG3gFmbjhHSVAMKHp0XKW7RavyqdoNYBU16x0Xvxz63EmCrtc2HM5Fx_ehxPqXv7pvi-jmI7lEtXEOBDqvgsjnImR82d9scXL1qThpvoAbTAxlMR2hkBNc_4tFR1tiHbRXyXQuVYm4nwxf96mEZSc2mlPQsVlgz8FaLDyaHqKmoyaFNhVUPE_WalNJT65OFyESEPk8cjfA53U4LrzZ_RhPlXUaFO8TwBkwOc7JncWw3voG4wvXvUH6",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 29.316957,
                    "lng": 117.22715
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.31777082989272,
                        "lng": 117.2283681798927
                    },
                    "southwest": {
                        "lat": 29.31507117010728,
                        "

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAB_w1VALrvG08C5uPjP3IZOsYSkHan9eCyEfoprK2nLVAEI4xkTVySlFp-ulMxU8KuYIvpi2SEuioB9HpnGmjsu_JgZ4kS_iQ5G48V9AL9vdxOIWkU0yQHYS0eF8HNsKVgDokgTpUQW5J1L7QMthJA3M0iySAgUyj5xaDRFiCzZtt5rIE4VLVSNcpXHRSUysAxBbfZ3TraA4gKHPt5MGpa6xokF666hUy0druXQQ-xS3Qn1EQwaiQQ_y8bSfMa9AfhzTFjAmfo1r-kCw1Z2FaQ1D-YjkrjSLDlunNKLEYoeET607G5faB2OcGz9PBLYpLRjrJlYtvCWIkHbX01WWyRPRZoPcpujkfCBR6g-M7-xpWteVEGAxSXAYjhAmgsJyvtWpQBE4LAoNXkRuzLZs610SEGniQHoH5FKeTsp3sIW4HuAaFBnobhEyppvHHVOUsP-w727k05-j",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 23.365841,
                    "lng": 116.704035
                },
                "viewport": {
                    "northeast": {
                        "lat": 23.36725037989272,
                        "lng": 116.7051766798927
                    },
                    "southwest": {
                        "lat": 23.36455072010727,
                        

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAD9dXk8Zoz5wM5TdUHdtRyPTEgs7YvlBxVI9uKPAjyBFVOlOC2EZ8UlClfX6aI85oLfaAV1SmoeGSsDx3gd1Lkjwv34F9wlz3YXYfI01AlKA2_NCR3Roc1eottuxqIGOd855xVAeN35tP2myC-Yk5twkehjl3W23EKp99rfLauUV0ONLArsJz9bH7y97yXn7OMz-g7fHnjLGIxFRMkPwxC5T_dOVXkU6n-jBSqazDcv8QDlW6EkmrRC-ewUpBFuOkMNhkoGbHKO0BYMKt9FHbHI2RsFAc7qt6mwbFGCnLzUiInoJnFW6IyEbD9Kf9ezLUJAB0KJRBNNKUA02Rmvv4Rd9LV44iQNQ9_XKgnCO_QKk5VHc91zyeBO0BWDdBIiB6Yg9Hx0_A2wKK_wHKjRo2YUSEEO0YowRsgmgccZMtjUTRfwaFPUr5_WTElJ5hSaPY1W6ULoYTk03",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -31.9604656,
                    "lng": 141.4609665
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.95926892010728,
                        "lng": 141.4625679298927
                    },
                    "southwest": {
                        "lat": -31.96196857989272,
                   

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAH90UbVzkHVhCLUknxcAzyObDQ2jPmgcAO4pqX4sPe8wa9XdVOBT5WYZIzVzoZGoDwtlWqKw1Ug-tYMdRqvwGO_lPjg362EDH97veob_J561iSBXN7Zfs8gZzYxKCaEygFJc6elUnEb3Yy6OsQY-hDNSpsR0Hnrg8nMkxP_0Rn0beLqTF6pex3UP7xaCZ4R00jD0f_WfKb2Fj7i9E5yCCq5wKV-pkoTyGZp1_MOVDQ1XoocVLUP_PK8tKyqhD9EcsVkrIck_e_rnTOcVvUpOnkLx7C-YDYaPmd2ftJAI1dscVhPZPoW04lvXQy5PpCYDCipoqK3x562EjXWRfmPHHebbl5zTF1oiApwqDsW529Xvv0Yn5tzhliMUC_0wZapL068mNeW_0XgDY9zakAUR6lMSEOQM8lssAMwrYtXQcuTxOywaFPQgcUctpLRkqUSKEQVzsupriFB4",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -32.2180258,
                    "lng": -58.13374389999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.21668657010728,
                        "lng": -58.13238322010728
                    },
                    "southwest": {
                        "lat": -32.21938622989272,
           

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAJ1hRGEDFZ5UxU27tesw2oUiG8QV-Rq9pBmUFMmC1Khx1u7s0TRFwNGNbHiD4us6J1dBL_UcYmoFL3QWxFpDvJr7aXsgVvC9bT_9HJ358lgfj8dl8aEnO7AgzoysE5Vr2ag6oP0f1Wza6gOhUsbn7yexcD-LSh1_CRvCz-w_AkGOLGNnUHlsX8-9ERHZgIldH5QAgtXYnv6Y1eTmPf5skt_h1XNvQhSbXBlBCODxjTHWxrRNY1me41Pkm9bVKmYliQXK6B8FPYtFduKbhqdQOV2RhSPLNILJ6mT2nMG4uN44K75n8vx_GKWUl42s5tWuvkHFKFAkLtL7yMyb8h4zOw0Ga7Pbrfbtx_6qZBOQTgjPYnjG5voTYyQsBukB0SPSZ5kZVr6Wq8QD801ii1S-dGsSEI3F1QmB4KuDuwFrhhE_0q4aFAg9H49JfPDNJdj3R1vMPTckPx4H",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 48.6226593,
                    "lng": 22.9716452
                },
                "viewport": {
                    "northeast": {
                        "lat": 48.62401317989272,
                        "lng": 22.97287707989272
                    },
                    "southwest": {
                        "lat": 48.62131352010728,
                       

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAJXtypBwCAPbzXtcEmnANNS9z-td-OaOzotRYnov6GUmJL-8qtd_EJJTZOLRNo0_t6LtdHAnXy_2SaOY-VfpMO52SlxuiLKh-b5p7kI3oI8CdBhRMbHAI5DnsDkTMt0qUJfOHr9Y3arPiJOvA4u3UWxx4xN8qzZp0nBvGuTAoH-SnRTXY1F2vk7ulLY7cUpS66yGPVr802hn9mCWaPYX4Tkplo5tliR0GldUU8_yw7GnF8XPd5Oftjxn6KM00j_8lEyDvM-YMTIFxohotZly4L0DObvBm8jv3gdH0wsUGvKO-HVNUxUKTzPCuwdeUjnOudnA_-nxE2n9lcI96zgp2AJBX3JFgjQF0KOaTkeI9BXTQlU5ZoYldBenLVmRJOu19of5jpUf5gRVv0kAjQQWfHYSECE_HYnjKsOZIUfbiYyCkkcaFAPPK8S1SCpGMrPa-6d4GYxZVJGB",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 45.9379741,
                    "lng": -0.9587920999999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 45.93932982989272,
                        "lng": -0.9574918701072778
                    },
                    "southwest": {
                        "lat": 45.93663017010728,
            

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAANI7UV4Csue75pVf-3kktVgAhBjV0OEzXB5lalEl5YpICbVogRpORrsU3bTxtCGHIjr_8YbFjKbFE9tFG3-dvEnmlZ4lA-zxRm1qJJ736UxWaOpYQilLoxSJcr8SIoy1JypTXab672ux5nPNfINnyKWBSxW_v7LVOTmWwAUlzoLKfaAZPIIfkmxnwzKIsZi0s-hwQjMaaYtjBxLMpvBUbhz4GXo0QBevZSlgyKje9Ghrj5VQC3Iq3g6F7nqb4qU_igibYhsgYwFThstw7Iv-1o1J-bGOdMCfEYTc-47yZaHy1eq1Nghd9nLBxcNjFu0bBLVH9aqLg7FYYznGa0i2H_RAnr1tn7N92XnS9NIzNByGaq2wyAdE-LJUE5v26pPFrtcidfZ3lW_rilVZH-66V6ESELnL5GjG1XMoliyzeuYAw7AaFKcVwDYD0RDyHVbXQuICgyZYpwNp",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -27.4748278,
                    "lng": -58.8407361
                },
                "viewport": {
                    "northeast": {
                        "lat": -27.47360607010728,
                        "lng": -58.83940597010729
                    },
                    "southwest": {
                        "lat": -27.47630572989272,
                  

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAJkrIFp4RalMmztWuaCZ5T2R-1lIGrakQM6RB2wo186Gj8RS5fQuKDG_ZL0s27An7tjISbimkFKoJktuzuvXpbFFkbcX2if7olC6wK_3ENdiZNR3swG6Rnuji_yrBQERPSiQdTTP5GoDIsfswBG5fEiOmsjqJZY8vxyrs5tSkSasvP3t7JML4a5f11oMwp42eP31Kmmb6lcDRp0qC_DTxn-HLC2l00GUDhWfNAxS8Q9VQEy64L8wxUNzRwc1wAbxz1neTIrZ_1c4SnsLCrdvtlOopcwL_e6mhKbqAGpaSVmrSC64mBK0qzIuiNHl3ZZPtXJYJ0bWb2qfRNh4qNYKG1Qf5q1Y14gfV3jmM01dYXlFA6FcbhyFtTP7lmE6uKSjwzReQIBEpb5rwP4J9fTo2agSEOwGxYRSMkzfgwZ2yjxc3LgaFL4g6z5-GPw2vdZW87MTsVNcNYPM",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -25.4660915,
                    "lng": -53.5810895
                },
                "viewport": {
                    "northeast": {
                        "lat": -25.46474532010727,
                        "lng": -53.57998972010728
                    },
                    "southwest": {
                        "lat": -25.46744497989271,
                  

,City ID,Date,Country,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloud Cover,Hotel Name
0,134,2020-04-09 13:48:39,CN,Jingdezhen,29.29,117.21,63.05,61,3.22,0,Fairfield by Marriott Jingdezhen
1,296,2020-04-09 13:49:43,CN,Chaozhou,23.67,116.64,66.20,63,4.47,0,Shantou International Hotel
2,331,2020-04-09 13:49:46,AU,Broken Hill,-31.95,141.43,60.80,59,3.36,0,ibis Styles Broken Hill
3,365,2020-04-09 13:49:50,AR,Concepción del Uruguay,-32.48,-58.24,64.00,64,1.99,0,Costarenas
4,402,2020-04-09 13:49:53,UA,Poroshkovo,48.67,22.75,73.40,17,2.24,0,Polyansky castle
5,416,2020-04-09 13:49:54,FR,Arrondissement de Saintes,45.67,-0.75,75.99,25,4.70,0,"L'Esprit du 8, chambres d'hôtes"
6,451,2020-04-09 13:49:57,PY,General José Eduvigis Díaz,-27.17,-58.42,69.01,55,4.00,0,La Alondra Casa de Huéspedes
7,560,2020-04-09 13:50:06,BR,Realeza,-25.77,-53.53,76.77,28,2.62,0,Hotel Golden View


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))